<img width="8%" alt="Forecast.png" src="https://raw.githubusercontent.com/jupyter-naas/awesome-notebooks/master/.github/assets/logos/Forecast.png" style="border-radius: 15%">

# Forecast - List all clients
<a href="https://bit.ly/3JyWIk6">Give Feedback</a> | <a href="https://github.com/jupyter-naas/awesome-notebooks/issues/new?assignees=&labels=bug&template=bug_report.md&title=Forecast+-+List+all+clients:+Error+short+description">Bug report</a>

**Tags:** #forecast #clients #api #list #python #get

**Author:** [Landry Christensen](https://github.com/lchristensen6)

**Last update:** 2023-08-08 (Created: 2023-08-08)

**Description:** This notebook will list all clients from the forecast API. Forecast is a service that connects to harvest and allows you to plan for allocations to harvest projects.

**References:**
- [Forecast Help Center](https://help.getharvest.com/forecast/faqs/faq-list/api/)


<div class="alert alert-info" role="info" style="margin: 10px">
<b>Disclaimer:</b><br>
Kindly be aware that the Forecast API is currently not available for public use or documentation. If you wish to customize and utilize this template for your specific needs, please reach out to the author.
<br>
</div>

## Input

### Import libraries

In [ ]:
import requests
import pandas as pd
import naas

### Setup variables
**Mandatory**

- `account_id`: Account ID from Forecast
- `client_id`: Client ID from Forecast
- `client_secret`: Client Secret from Forecast
- `refresh_token`: Refresh Token from Forecast

**Optional**
- `limit`: clients limit, to get all clients enter -1

In [ ]:
# Mandatory
account_id = naas.secret.get("FORECAST_ACCOUNT_ID") or "YOUR_FORECAST_ACCOUNT_ID"
client_id = naas.secret.get("FORECAST_CLIENT_ID") or "YOUR_FORECAST_CLIENT_ID"
client_secret = naas.secret.get("FORECAST_CLIENT_SECRET") or "YOUR_FORECAST_CLIENT_SECRET"
refresh_token = naas.secret.get("FORECAST_REFRESH_TOKEN") or "YOUR_FORECAST_REFRESH_TOKEN"

# Optional
limit = 1000

## Model

### List all clients

This function will list all clients from the Forecast API

In [ ]:
# Get Harvest Access token
def get_access_token():
    res = requests.post(
        url="https://id.getharvest.com/api/v2/oauth2/token",
        data={
            "client_id": client_id,
            "client_secret": client_secret,
            "refresh_token": refresh_token,
            "grant_type": "refresh_token",
        },
        headers={"User-Agent": 'Harvest-Scraper'},
    )
    res_json = res.json()
    if res.status_code == 201:
        return res_json["access_token"]
    else:
        print(res.status_code, res.json().get("message"))
        return ""
    
def list_clients(
    account_id,
    access_token,
    limit=-1,
):
    # Init
    data = []
    df = pd.DataFrame()
    
    # Requests
    url = f"https://api.forecastapp.com/clients"
    headers = {
        "Accept": "application/json",
        "Forecast-Account-ID": account_id,
        "Authorization": "Bearer " + access_token,
    }
    
    res = requests.get(url=url, headers=headers)
    while True:
        if res.status_code == 200:
            # Get data
            res_json = res.json()

            clients = res_json.get("clients")
            for client in clients:
                data.append(client)

                # Manage limit
                if limit != -1 and len(data) >= limit:
                    break
            break
        else:
            print(res.status_code, res.json().get("message"))
            break

    # Transform in dataframes
    df = pd.DataFrame(data)
    return df

## Output

### Display result

In [ ]:
df_clients = list_clients(
    account_id,
    get_access_token(),
    limit
)
print("Row fetched:", len(df_clients))
df_clients.head()